In [1]:
import platform
print(platform.machine())


AMD64


In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 2060
Using device: cuda


In [3]:
print("CUDA Available:", torch.cuda.is_available())
print("Current GPU:", torch.cuda.current_device())
print("GPU Memory Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("GPU Memory Cached:", torch.cuda.memory_reserved() / 1024**2, "MB")

CUDA Available: True
Current GPU: 0
GPU Memory Allocated: 0.0 MB
GPU Memory Cached: 0.0 MB


In [4]:
import torch

# Create a tensor and move it to GPU
x = torch.rand(3, 3).to("cuda")

print("Tensor on GPU:", x)
print("GPU Memory Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("GPU Memory Cached:", torch.cuda.memory_reserved() / 1024**2, "MB")

Tensor on GPU: tensor([[0.1331, 0.8529, 0.1293],
        [0.3817, 0.6938, 0.1101],
        [0.6866, 0.7192, 0.0525]], device='cuda:0')
GPU Memory Allocated: 0.00048828125 MB
GPU Memory Cached: 2.0 MB


In [2]:
import torchvision
import torch.nn as nn

class Identity(nn.Module):
    def __init__(self):
        super(Identity,self).__init__()

    def forward(self,x):
        return x

model = torchvision.models.vgg16(
    pretrained = True,
)

for param in model.parameters():
    param.requires_grad = False

model.avgpool = Identity()
model.classifier =nn.Linear(512,10)

print(model)

c:\Users\jarvi\OneDrive\Desktop\tehwei\github\MLOPs-Project---DL\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jarvi\OneDrive\Desktop\tehwei\github\MLOPs-Project---DL\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
# Modify the first conv layer to accept 1-channel (grayscale) or 4-channel (RGBA)
new_in_channels = 3  # Change to 4 if you have RGBA input
old_conv1 = model.features[0]  # First convolution layer

# Create a new conv layer with modified input channels
new_conv1 = nn.Conv2d(in_channels=new_in_channels, 
                       out_channels=old_conv1.out_channels, 
                       kernel_size=old_conv1.kernel_size, 
                       stride=old_conv1.stride, 
                       padding=old_conv1.padding)

# Copy the weights from the old layer (for the first 3 channels, if applicable)
if new_in_channels == 1:
    new_conv1.weight.data = old_conv1.weight.data.mean(dim=1, keepdim=True)  # Convert RGB to Grayscale
elif new_in_channels == 4:
    new_conv1.weight.data[:, :3, :, :] = old_conv1.weight.data  # Copy RGB weights
    new_conv1.weight.data[:, 3:, :, :] = old_conv1.weight.data[:, :1, :, :]  # Initialize 4th channel with Red weights

# Replace the old conv1 layer with the new one
model.features[0] = new_conv1

print(model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1